# Part 2: Color

Now that we have the basics of image manipulation under our belt, let's take things to the next dimension: color images!

In [ ]:
%matplotlib inline
import numpy as np
import scipy.signal
import skimage.color
import cv2

from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from matplotlib import collections
from mpl_toolkits.mplot3d import art3d

print("Import done!")

! git clone https://github.com/Tino-FRC-2473/CVTraining repo_files

## 2.1 What is color, really?

In elementary school, we are taught that colored light is made up of three primary colors that we can mix together to make any other color. This is often shown in a kind of color venn diagram or color wheel

![Color wheel](https://upload.wikimedia.org/wikipedia/commons/thumb/c/c2/AdditiveColor.svg/240px-AdditiveColor.svg.png)

In physics, you probably learned that the color of light is dependent on the frequency of light waves. This is usually demonstrated by a prism breaking light into a continuous rainbow as different frequencies of light are bent differently.

![Visible Spectrum](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d9/Linear_visible_spectrum.svg/500px-Linear_visible_spectrum.svg.png)

But there's a bit of a problem here. Just putting some red and green photons together doesn't change their frequency to yellow. And a cyan photon can't magically split apart into a blue and green photon when it enters a camera. And why are red, green, and blue so special?

So we come to the next generation of color theory: the tristimulus model. The light sensing cone cells in our eyes have three different kinds of pigments. Each pigment produces a different level of response when it absorbs different frequencies of light.

![Cone color response](https://upload.wikimedia.org/wikipedia/commons/thumb/1/1e/Cones_SMJ2_E.svg/500px-Cones_SMJ2_E.svg.png)

Our brains then combine the level of response from each of the three classes of cone cells to estimate what frequency of light we're looking at. For camera and display makers, this is great news since if we can just stimulate these three classes of cells independently, we can simulate any color we want. The peaks of the three responses are at red, green, and blue. So we have our first color system, RGB.

Note, however, that the three cells aren't completely independent. In other words, we can't pick frequencies that only stimulate one of the three cell types because their response curves overlap. So we can mix red and green light to look *similar* to yellow light, but not *exactly the same* as true yellow light.

On top of this, the brain does all kinds of processing to estimate color that we just can't model with three numbers. For instance, the color brown doesn't appear on any of these models, because it is an artifact of the way our brains judge relative brightness. If you're interested in learning more, the famous book [_Interactions of Color_](hhttps://sccl.bibliocommons.com/item/show/389841118) by Josef Albers lays out a series of exercises to explore how we perceive color with an eye toward graphic design and art.

![Albers](https://collectionapi.metmuseum.org/api/collection/v1/iiif/737721/1626557/restricted)

We're only scratching the surface here, but I hope this gives a bit of background into where RGB comes from and the underlying complexity of color representation.

![XKCD](https://imgs.xkcd.com/comics/color_models.png)

## 2.2 Color images

So to summarize, we need three values in order to represent color as humans see it. So a color image has three values for each pixel. These are usually referred to as channels, so for RGB we have a red channel, a green channel, and a blue channel. 

Now remember, OpenCV does things a bit backward, so we're going to need to swap the order of the color channels from BGR to RGB.

In [ ]:
def bgr_to_rgb(image):
  return image[:,:,::-1]

! curl https://upload.wikimedia.org/wikipedia/commons/thumb/c/c2/AdditiveColor.svg/240px-AdditiveColor.svg.png -o rgb_diagram.png
! curl https://www.nasa.gov/sites/default/files/thumbnails/image/pillars_of_creation.jpg -o hubble.jpg

color_diagram_image = bgr_to_rgb(cv2.imread('rgb_diagram.png'))
plt.imshow(color_diagram_image)
plt.show()

hubble_image = bgr_to_rgb(cv2.imread('hubble.jpg'))
plt.imshow(hubble_image)
plt.show()

### Exercise
When looking at a numpy array representing a color image, the color channels make up a third dimension. So instead of a one-dimensional scalar brightness like we had in grayscale images, we have three values representing each color channel. Let's split these channels apart and take a look. I've written the code to isolate the red channel. Fill in the code for isolating the green and blue channesl.

In [ ]:
plt.figure()
plt.title("Red only")
red_only_image = color_diagram_image.copy()
# Zero out Green channel
red_only_image[:,:,1] = 0
# Zero out Blue channel
red_only_image[:,:,2] = 0
plt.imshow(red_only_image)
plt.show()

plt.figure()
plt.title("Green only")
green_only_image = color_diagram_image.copy()
# WRITE CODE HERE
# Zero out Red channel

# Zero out Blue channel

# END CODE
plt.imshow(green_only_image)
plt.show()

plt.figure()
plt.title("Blue only")
blue_only_image = color_diagram_image.copy()
# WRITE CODE HERE
# Zero out Red channel

# Zero out Green channel

# END CODE

plt.imshow(blue_only_image)
plt.show()

##2.3 Color Spaces

While RGB matches how humans see the world, it often isn't very intuitivite or efficient to work with. In particular, it's often hard to separate what color a pixel is from how bright a pixel is. Is the blue channel high in this pixel because it's a picture of a blue object, or is it just a very bright pixel so all three channels have large values?

To solve these problems, there are a number of different schemes for representing color information known as "color spaces" each optimized for different tasks. Note that since color information requires three channels, most useful color spaces are also three dimensional. However what each dimension/channel represents can be remapped and redesigned in creative ways.

### 2.3.1 Hue-Saturation-Value (HSV)

![HSV Cylinder](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4e/HSV_color_solid_cylinder.png/640px-HSV_color_solid_cylinder.png)

One very common color space is the Hue-Saturation-Value color space, commonly abbreviated HSV. This color space is designed to present color in a way that more closely matches how we intuitiviely think about color. First, H represents the "hue" of a color as an angular value from 0-360º. This fits well with the idea of a color circle. Saturation (S) roughly corresponds to how intense or saturated a color is. Finally, value (V) is intended to represent the brighness of a color, with 0 being black and 1 being as bright as possible.

You'll often seen HSV being used in user interfaces for "color picker" tools. These components match well to the way we intuitively talk about color, such as "bright green" or "dark blue." However, there are some drawbacks for using HSV for computer vision. For instance, the difference between saturation and value is a bit ambiguous. Near the center of the HSV cylinder shown above, there's a big region of pale white colors that look about the same, even though their value and saturation are quite different. After all, what's the difference between a low-saturation or faded color, and a bright white? These ambiguities make HSV tricky to use in computer vision algorithms that need to work across different lighting conditions.



### Exercise
Let's have a little fun with HSV. One popular kind of filter simply boosts or decreases the saturation of a image. Let's implement that now. [Skimage's color module](https://scikit-image.org/docs/dev/api/skimage.color.html) lets us covert between color spaces. The second argument is a constant describing the source and destination color space names.



In [ ]:
def edit_saturation(rgb_image, factor):
  # Convert RGB to HSV
  hsv_image = skimage.color.rgb2hsv(rgb_image)
  # Multiply the saturation channel of hsv_image by factor
  # WRITE CODE HERE

  # END CODE
  # Convert back to RGB
  return skimage.color.hsv2rgb(hsv_image)

Now let's use this function to adjust the saturation of a test image

In [ ]:
plt.figure()
plt.title("Original")
plt.imshow(hubble_image)

plt.figure()
plt.title("Saturation boost")
plt.imshow(edit_saturation(hubble_image, 2))
plt.show()


###2.3.2 Luminance and chrominance, YUV/YCbCr

This problem of isolating lighting information from color information comes up in a wide range of different applications, and so it is no surprise that there are other color spaces that perform this kind of separation. In particular, the closely related YUV and YCbCr color spaces were designed to explicitly separate brightness from color. These color spaces were originally designed when the first color televisions were being developed, as a way to separate a brightness-only signal for older black and white televisions from a color television signal. These days they are commonly used in computer vision and digital video processing applications where color and brightness need to be treated differently.

The basic idea is that all the perceived brightness information is placed in the first "luminance" channel, abbreviated "Y", while the color information is placed in the other two "chrominance" channels, U/V or Cb/Cr. U or Cb can be thought of as measuring the "blueness" of a pixel, with 1 being maximum blue, and 0 being the opposite of blue (basically yellow). V or Cr can be thought of as measuring the "redness" of a pixel, with 1 being maximum red, and 0 being the opposite of red (i.e. green).

![CbCr plane](https://upload.wikimedia.org/wikipedia/commons/thumb/3/34/YCbCr-CbCr_Scaled_Y50.png/480px-YCbCr-CbCr_Scaled_Y50.png)

This is useful because the Y channel on its own effectively represents a grayscale version of a color image. This can be useful for color-independent processing, such as if you want to find the shadows or highlights in a color image. Meanwhile, looking at the U/V or Cb/Cr channels alone lets you pick apart different colors in a picture very easily. This is used extensively in color grading movies, and the use of these "chroma" channels leads to the popular "chroma-keying" or green-screen proceses used in video production and visual effects. We'll explore this more in the next section.



### Exercise

To help visualize YUV, let's take an image and break it apart into its Y, U, and V components. Note that when visualizaing the U and V channels, zeroing out Y channel would just turn the image black. So let's set Y to a fixed value of 0.5, representing half the maximum luminance.

In [ ]:
yuv_image = skimage.color.rgb2yuv(hubble_image)

plt.figure()
plt.title("Y only")
y_only_image = yuv_image.copy()
# Zero out U channel
y_only_image[:,:,1] = 0
# Zero out V channel
y_only_image[:,:,2] = 0
plt.imshow(skimage.color.yuv2rgb(y_only_image).clip(0, 1))
plt.show()

plt.figure()
plt.title("U only")
u_only_image = yuv_image.copy()
# WRITE CODE HERE
# Set Y channel to half luminance (0.5)

# Zero out V channel

# END CODE
plt.imshow(skimage.color.yuv2rgb(u_only_image).clip(0, 1))
plt.show()

plt.figure()
plt.title("V only")
v_only_image = yuv_image.copy()
# WRITE CODE HERE
# Set Y channel to half luminance (0.5)

# Zero out U channel

# END CODE
plt.imshow(skimage.color.yuv2rgb(v_only_image).clip(0, 1))
plt.show()

## 2.4 Color Segmentation

One of the most basic tasks in computer vision is figuring out where interesting things are in the image. For instance, a robot might be looking for a target to shoot a ball into or game pieces that it can pick up from the field. This general task of classifying an image into different regions of interest is called image segmentation. The idea is to label each pixel into different classes or segments, each of which belong to a different object of interest or share some useful property.

Given how important image segmentation is for computer vision algorithms, it isn't surprising that there exists a wide range of different algorithms covering all kinds of different use cases and techniques. In this section, we will focus on one of the most simple yet effective techniques for image segmentation commonly used today: color based segmentation.

The basic idea here is really quite simple. If you know what color the object you are looking for should be, then you can divide the image into pixels that match that color and pixels that don't. The tricky part is turning this into an algorithm that is both accurate and robust to different lighting conditions.

For the coding exercises, we'll be using color segmentation to perform background subtraction. Commonly known as "chroma-keying" or "green screen," the task is to identify the background pixels of a video which has a well defined background color. Then, we can remove only the background pixels and replace them with some other, more interesting background.

Foreground image is from "[#INTRODUCTIONS" By LaBeouf, Rönkkö & Turner](https://vimeo.com/125095515), used under CC BY-NC-SA licence. Feel free to add your own background image, but make sure to resize it to 640x360 pixels.

In [ ]:
foreground_image = bgr_to_rgb(cv2.imread('repo_files/data/foreground.png'))
plt.figure()
plt.title("Foreground")
plt.imshow(foreground_image)

background_image = bgr_to_rgb(cv2.imread('repo_files/data/background.png'))
plt.figure()
plt.title("Background")
plt.imshow(background_image)
plt.show()

### 2.4.1 Background color model

The first step is to come up with a model for what we will consider the background color. First, to avoid issues with varying lighting, let's convert to YUV and only consider the U and V channels. Then, we'll select a section of the image that we know is background (say, the left third of the image) and find the mean and standard deviation of the U and V values in that region. From these statistics, we can model the green color of the background as some mean U,V value, plus/minus some margin of error.

### Exercise

In [ ]:
# Convert foreground_image to YUV
foreground_yuv = # WRITE CODE HERE

# Extract known green pixels, from the left 200 columns of pixels
plt.title("Known green pixels")
plt.imshow(foreground_image[:,:200])

# Convert the known green pixels to YUV
known_background_yuv = # WRITE CODE HERE

# Reshape into list of pixels, with dimensions (length * width, 3)
known_background_yuv = np.reshape(known_background_yuv, (-1, 3))

# Find mean and standard deviation. Hint: remember you can pass "axis" argument to only average along one dimension of an array
mean_color_yuv = # WRITE CODE HERE
stddev_color_yuv = # WRITE CODE HERE

# Display the mean color
plt.figure()
plt.title("Mean color")
plt.imshow(skimage.color.yuv2rgb(np.full((25, 25, 3), mean_color_yuv)))
plt.show()

print("U standard deviation:", stddev_color_yuv[1])
print("V standard deviation:", stddev_color_yuv[2])


### 2.4.2 Segmentation
Great, now we can perform actual segmentation. The output of this step will be a "mask" array. Each pixel of the mask is simply a integer 0 if that pixel is the same color as the background, or 1 if it isn't. Of course, we want to give some leeway for slight variations in color. So instead of doing a straight equality comparison, we check if the color matches within a tunable margin of error.

### Exercise

First, let's define our lower and upper thresholds, as some multiple of the standard deviations for U and V. Remember we want to ignore Y so we aren't affected by changes in lighting. So set the Y threshold to accept any value between 0 and 1.

In [ ]:
low_threshold = np.array([0.0,
                          mean_color_yuv[1] - 10 * stddev_color_yuv[1],
                          mean_color_yuv[2] - 10 * stddev_color_yuv[2]])

# WRITE CODE HERE
high_threshold = 
# END CODE
print("Low threshold:", low_threshold)
print("High threshold:", high_threshold)

Next let's perform the actual comparison. Remember that for boolean arrays, `a * b` gives the logical and of a and b. 

In [ ]:
# mask is True where both U and V are between our low and high thresholds
background_mask = np.all((foreground_yuv > low_threshold) * (foreground_yuv < high_threshold), axis=2)
# Covert mask to integer 1 where True and 0 where False
background_mask = np.where(background_mask, 1, 0)
# Tack on an extra dimension to make mask shape (360,640,1), so that we can do numpy ops with color images
background_mask = np.expand_dims(background_mask, 2)

plt.figure()
plt.title("Masked foreground")
plt.imshow(foreground_image * (1 - background_mask))

plt.figure()
plt.title("Masked background")
plt.imshow(background_image * background_mask)
plt.show()

Finally, combine the masked foreground with the masked background image.

In [ ]:
# Set pixels that are in the foreground (background_mask == 0) to the foreground pixel values
# Set pixels that are in the background (background_mask == 1) to the background pixel values
combined_image = foreground_image * (1 - background_mask) + background_image * background_mask

plt.imshow(combined_image)
plt.show()